# Importing libraries

In [2]:
import pandas as pd
import math
from collections import Counter
# import time as t
from tqdm import tqdm 

In [3]:
data = pd.read_csv('Breast_Cancer.csv')

# Computing distances

In [4]:
def compute_distances(point1, point2):
    """
        Age, survival months, regional node positive, regional node examined and Tumor size
        are the continuous variables. Euclidean distance is used for measuring similarity 
        between these variables.

        Race, Marital Status, T Stage, N Stage, 6th Stage, Defferentiated, Grade, A Stage, 
        Estrogen Status, Progesterone Status are the categorical values. Hamming distance 
        is used for measuring the similarity across these variables.
    """
    euclidean_distance = math.sqrt(
        (point1['Age'] - point2['Age'] ) **2+
        (point1['Tumor Size'] - point2['Tumor Size'] ) **2+
        (point1['Regional Node Examined'] - point2['Regional Node Examined']) **2+
        (point1['Regional Node Positive'] - point2['Regional Node Positive'] ) **2+
        (point1['Survival Months'] - point2['Survival Months'])**2
    )

    # print("ed: ", euclidean_distance)

    

    hamming_distance = (
        (0 if point1['Race']==point2['Race'] else 1) + 
        (0 if point1['Marital Status']==point2['Marital Status'] else 1) +
        (0 if point1['T Stage']==point2['T Stage'] else 1) +
        (0 if point1['N Stage']==point2['N Stage'] else 1) +
        (0 if point1['6th Stage']==point2['6th Stage'] else 1) +
        (0 if point1['differentiate']==point2['differentiate'] else 1) +
        (0 if point1['Grade']==point2['Grade'] else 1) +
        (0 if point1['A Stage']==point2['A Stage'] else 1) +
        (0 if point1['Estrogen Status']==point2['Estrogen Status'] else 1) +
        (0 if point1['Progesterone Status']==point2['Progesterone Status'] else 1) 
    )
    
    
    # print("hd: ", hamming_distance)
    return euclidean_distance + hamming_distance

# Splitting dataset into train, val and test sets

In [5]:
def split_dataset(data):
    totalRows = data.shape[0] - 1

    """
        split data into train, validation and testing sets : 75-15-15% each
        find the total size of the dataset and *0.75, .15, .15
    """

    train_boundary = math.floor(0.70*totalRows)
    val_boundary = train_boundary + math.ceil(0.15*totalRows)
    test_boundary = val_boundary + math.ceil(0.15*totalRows)

    train_data = data.iloc[:train_boundary]
    val_data = data.iloc[train_boundary:val_boundary]
    test_data = data.iloc[val_boundary:test_boundary]

    train_Y = train_data['Status']
    train_X = train_data.drop(['Status'], axis=1)

    val_Y = val_data['Status']
    val_X = val_data.drop(['Status'], axis=1)


    test_Y = test_data['Status']
    test_X = test_data.drop(['Status'], axis=1)

    # print(train_X.shape[0])
    # print(val_X.shape[0])

    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [6]:
train_X, train_Y, val_X, val_Y, test_X, test_Y = split_dataset(data=data)

# Given a value of K, what are the predictions 
# for the 'Status' label on the val set

In [7]:
def parameter_tuning_knn(data, k):
    
    train_X, train_Y, val_X, val_Y, test_X, test_Y = split_dataset(data=data)

    val_pred = {}
    point_distance_map = {}

    point_point_map = {}

    for val_index in tqdm(range(list(val_X.shape)[0])):

        """
            for every point in the validation dataset, find the k nearest neighbours by computing distances,
            map and store distances to its training point, sort the map, and look at the first k points.
        """


        # print("Iterating at val_point: ", val_index)

        for train_index in range(list(train_X.shape)[0]):
            # print("Iterating at train_point: ", train_index)
            distance_list=[]
            point_list=[]
            # print(val_index, train_index)
            distance = compute_distances(val_X.iloc[val_index], train_X.iloc[train_index])
            point_distance_map[train_index] = distance

    
        sorted_distances_point_map = dict(sorted(point_distance_map.items(), key=lambda item: item[1]))
        
        """
            while looking at the first k points, find the most occuring 'Status' value among them, using
            train_Y and report it as the output for that val_point
        """


        counter = 0
        output = []
        for pair in sorted_distances_point_map.items():
            if(counter>=k): break                           # already found K neighbours
            # print(pair)
            point_number, distance = pair
            output.append(train_Y.iloc[point_number])
            # print(train_Y.iloc[point_number])
            counter+=1
        
        pred_status, trash = Counter(output).most_common()[0]
        # print(pred_status)
        val_pred[val_index]=pred_status
        
    return val_pred

# Testing

In [7]:
val_pred = parameter_tuning_knn(data, 3)
val_Y
validation_predictions = {}
for pair in val_pred.items():
    validation_predictions[pair[0]+2816]=pair[1]


validation_predictions
nocp = 0
for point in validation_predictions:
    if validation_predictions[point] == val_Y[point]:
        nocp+=1
        
accuracy_3 = nocp/604
list_of_accuracies = {}
len(val_pred)

100%|██████████| 604/604 [01:58<00:00,  5.09it/s]


# How to compute accuracies 

tp, tn, fp, fn 

if prediction is correct:
    actual + , pred + : tp++
    actual - , pred - : tn--
if prediction is wrong:
    actual + , pred - : fn++
    actual - , pred + : fp++
    

balanced accuracy : 
    for alive class:
        how many were classified as alive       tp
      -------------------------------------- = -----
          how many were actually alive         tp+fn
    for dead class:
        how many were classified as dead        tn
      -------------------------------------- = -----
          how many were actually dead          tn+fp

           2tp
f1 = ---------------
      2tp + fp + fn 

# Testing

In [24]:
tp = {}
fp = {}
tn = {}
fn = {}
k = 1
tp.setdefault(k,0)
tn.setdefault(k,0)
fn.setdefault(k,0)
fp.setdefault(k,0)

for point in validation_predictions:    
    if validation_predictions[point] == val_Y[point]:
        if val_Y[point] == 'Alive': 
            tp[k]+=1
        else:
            tn[k]+=1
    else:
        if val_Y[point] == 'Alive':
            fn[k]+=1
        else:
            fp[k]+=1  

tp, fp, tn, fn

# Running KNN for different K's and computing accuracies

In [8]:
list_of_accuracies = {}
list_of_balanced_accuracies = {}
list_of_f1_scores = {}

tp = {}
fp = {}
tn = {}
fn = {}



for k in tqdm([1,3,5,7,9,11], disable=True):
    print("Running for k = ", k)
    tp.setdefault(k,0)
    tn.setdefault(k,0)
    fn.setdefault(k,0)
    fp.setdefault(k,0)
    val_pred = parameter_tuning_knn(data, k)
    
    validation_predictions = {}
    for pair in val_pred.items():
        validation_predictions[pair[0]+2816]=pair[1]

    nocp = 0
    for point in validation_predictions:
        if validation_predictions[point] == val_Y[point]:
            nocp+=1

        if validation_predictions[point] == val_Y[point]:
            if val_Y[point] == 'Alive': 
                    tp[k]+=1
            else:
                tn[k]+=1
        else:
            if val_Y[point] == 'Alive':
                    fn[k]+=1
            else:
                fp[k]+=1
    list_of_accuracies[k] = nocp/len(val_pred)

    recall_1 = tp[k]/(tp[k]+fn[k])
    recall_2 = tn[k]/(tn[k]+fp[k])
    list_of_balanced_accuracies[k] = 0.5*(recall_1 + recall_2)

    list_of_f1_scores[k] = 2*tp[k] / (2*tp[k] + fp[k] + fn[k])

Running for k =  1


100%|██████████| 604/604 [01:58<00:00,  5.09it/s]


Running for k =  3


100%|██████████| 604/604 [01:58<00:00,  5.08it/s]


Running for k =  5


...

100%|██████████| 604/604 [01:58<00:00,  5.09it/s]


Running for k =  9


100%|██████████| 604/604 [02:00<00:00,  5.02it/s]


Running for k =  11


100%|██████████| 604/604 [01:58<00:00,  5.08it/s]


In [10]:
print(list_of_accuracies)
print(list_of_balanced_accuracies)
print(list_of_f1_scores)

{1: 0.8576158940397351, 3: 0.8973509933774835, 5: 0.9089403973509934, 7: 0.8940397350993378, 9: 0.9006622516556292, 11: 0.9056291390728477}
{1: 0.7056330046469454, 3: 0.7385923155389323, 5: 0.7404170916921682, 7: 0.7071517624390797, 9: 0.7208432505950357, 11: 0.7237334240054403}
{1: 0.9171483622350675, 3: 0.941398865784499, 5: 0.9484536082474226, 7: 0.9400749063670412, 9: 0.9438202247191011, 11: 0.9467787114845938}


In [15]:
def run_test_set(test_X, k):

    # train_X, train_Y, val_X, val_Y, test_X, test_Y = split_dataset(data=data)
    
    test_pred = {}
    point_distance_map = {}

    for test_index in tqdm(range(test_X.shape[0])):
        for train_index in range(train_X.shape[0]):
            distance = compute_distances(test_X.iloc[test_index], train_X.iloc[train_index])
            point_distance_map[train_index] = distance
        sorted_distances_point_map = dict(sorted(point_distance_map.items(), key=lambda item: item[1]))

        counter = 0
        output = []
        for pair in sorted_distances_point_map.items():
            if(counter>k): break
            point_number, distance = pair
            output.append(train_Y.iloc[point_number])

            counter+=1
        pred_status, trash = Counter(output).most_common()[0]
        test_pred[test_index] = pred_status
    
    return test_pred

In [17]:
test_X

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Regional Node Positive,Survival Months
3420,53,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Positive,6,3,63
3421,65,White,Married,T1,N1,IIA,Moderately differentiated,2,Regional,15,Positive,Positive,7,2,49
3422,53,White,Married,T1,N1,IIA,Well differentiated,1,Regional,15,Positive,Positive,2,1,69
3423,57,White,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,25,Positive,Positive,12,2,74
3424,53,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,13,Positive,Negative,17,2,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,Regional,9,Positive,Positive,1,1,49
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,Regional,46,Positive,Positive,14,8,69
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Negative,11,3,69
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,44,Positive,Positive,11,1,72


In [19]:
test_acc, test_bAcc, test_f1 = 0, 0, 0
test_pred = run_test_set(test_X, k)

test_predictions = {}
for pair in val_pred.items():
    test_predictions[pair[0]+3420]=pair[1]
    

100%|██████████| 604/604 [01:59<00:00,  5.06it/s]


In [23]:
test_Y

3420    Alive
3421    Alive
3422    Alive
3423    Alive
3424    Alive
        ...  
4019    Alive
4020    Alive
4021    Alive
4022    Alive
4023    Alive
Name: Status, Length: 604, dtype: object

In [22]:
for point in test_predictions:
    if test_predictions[point] == test_Y[point]:
        

(3420, 'Alive')
(3421, 'Alive')
(3422, 'Alive')
(3423, 'Dead')
(3424, 'Alive')
(3425, 'Alive')
(3426, 'Alive')
(3427, 'Alive')
(3428, 'Alive')
(3429, 'Alive')
(3430, 'Alive')
(3431, 'Alive')
(3432, 'Alive')
(3433, 'Alive')
(3434, 'Alive')
(3435, 'Alive')
(3436, 'Alive')
(3437, 'Alive')
(3438, 'Alive')
(3439, 'Alive')
(3440, 'Alive')
(3441, 'Dead')
(3442, 'Alive')
(3443, 'Alive')
(3444, 'Alive')
(3445, 'Alive')
(3446, 'Alive')
(3447, 'Alive')
(3448, 'Alive')
(3449, 'Alive')
(3450, 'Alive')
(3451, 'Alive')
(3452, 'Alive')
(3453, 'Alive')
(3454, 'Alive')
(3455, 'Alive')
(3456, 'Dead')
(3457, 'Alive')
(3458, 'Alive')
(3459, 'Alive')
(3460, 'Alive')
(3461, 'Alive')
(3462, 'Alive')
(3463, 'Alive')
(3464, 'Alive')
(3465, 'Alive')
(3466, 'Alive')
(3467, 'Alive')
(3468, 'Alive')
(3469, 'Alive')
(3470, 'Alive')
(3471, 'Alive')
(3472, 'Alive')
(3473, 'Alive')
(3474, 'Alive')
(3475, 'Alive')
(3476, 'Dead')
(3477, 'Alive')
(3478, 'Alive')
(3479, 'Alive')
(3480, 'Alive')
(3481, 'Alive')
(3482, 'Aliv

# Testing code

In [28]:
list_of_accuracies
tp, fp, tn, fn
list_of_balanced_accuracies = {}


for k in [1,3,5,7,9,11]:
    recall_1 = tp[k]/(tp[k]+fn[k])
    recall_2 = tn[k]/(tn[k]+fp[k])
    list_of_balanced_accuracies[k] = 0.5*(recall_1 + recall_2)
list_of_balanced_accuracies
"""

           2tp
f1 = ---------------
      2tp + fp + fn 

"""
list_of_f1_scores = {}
for k in [1,3,5,9,11]:
    list_of_f1_scores[k] = 2*tp[k] / (2*tp[k] + fp[k] + fn[k])
list_of_f1_scores

{1: 0.8576158940397351,
 3: 0.8973509933774835,
 5: 0.9089403973509934,
 7: 0.8940397350993378,
 9: 0.9006622516556292,
 11: 0.9056291390728477}

Based on Acc, BAcc and F1 scores, the optimal K- values are:
    1. Based on Acc         : 5
    2. Based on Bacc        : 5
    3. Based on F1 scores   : 5

In [ ]:
tp_test = {}
tn_test = {}
fp_test = {}
fn_test = {}

In [ ]:
tp.setdefault(k,0)
tn.setdefault(k,0)
fn.setdefault(k,0)
fp.setdefault(k,0)
test_pred = parameter_tuning_knn(data, k)

test_predictions = {}
for pair in test_pred.items():
    test_predictions[pair[0]+2816]=pair[1]

nocp = 0
for point in test_predictions:
    if test_predictions[point] == val_Y[point]:
        nocp+=1

    if test_predictions[point] == val_Y[point]:
        if val_Y[point] == 'Alive': 
                tp[k]+=1
        else:
            tn[k]+=1
    else:
        if val_Y[point] == 'Alive':
                fn[k]+=1
        else:
            fp[k]+=1
list_of_accuracies[k] = nocp/len(test_pred)